Задание 3. Проекционные методы для краевой задачи ОДУ второго порядка.

In [1]:
from dataclasses import dataclass
from math import sin
from typing import Callable, Literal

import numpy as np
import plotly.graph_objs as go
from numpy import linalg as LA
from numpy.polynomial import chebyshev as cheb
from numpy.polynomial.polynomial import Polynomial
from scipy import integrate
from scipy.sparse import lil_matrix
from scipy.special import jacobi

In [2]:
Function = Callable[[float], float]
Segment = tuple[float, float]
BoundaryCondition = tuple[float, float, float]
CoordinateSystem = list[Polynomial]


@dataclass
class BoundaryValueProblem:
    '''
    Forms of the equation:

    1: ``-(p(x)y')' + r(x)y = f(x)``,

    2: ``p(x)y'' + q(x)y' + r(x)y = f(x)``,

    3: ``-p(x)y'' + q(x)y' + r(x)y = f(x)``,

    ``p(x) ≥ p₀ > 0, r(x) > 0``.

    Boundary conditions:
    
    ``α₀y(a) - α₁y'(a) = α, |α₀| + |α₁| ≠ 0, α₀α₁ ≥ 0``,
    
    ``β₀y(b) + β₁y'(b) = β, |β₀| + |β₁| ≠ 0, β₀β₁ ≥ 0``
    '''
    form: Literal[1, 2, 3]
    p: Function
    q: Function
    r: Function
    f: Function
    segment: Segment
    left_condition: BoundaryCondition
    right_condition: BoundaryCondition

In [3]:
def get_jacobi(n: int, k: int) -> CoordinateSystem:
    return [jacobi(i, k, k) for i in range(n)]


def ritz_method(problem: BoundaryValueProblem, coordinates: CoordinateSystem) -> Function:
    assert problem.form == 1

    alpha_1, alpha_2, _ = problem.left_condition
    beta_1, beta_2, _ = problem.right_condition

    def bilinear_form(y: Polynomial, z: Polynomial) -> float:
        result = integrate.quad(lambda x: problem.p(x) * y.deriv()(x) * z.deriv()(x) + problem.r(x) * y(x) * z(x), -1, 1)[0]
        result += 0 if alpha_1 * alpha_2 == 0 else alpha_1 / alpha_2 * problem.p(-1) * y(-1) * z(-1)
        result += 0 if beta_1 * beta_2 == 0 else beta_1 / beta_2 * problem.p(1) * y(1) * z(1)
        return result
    
    def dot(phi: Function) -> float:
        return integrate.quad(lambda x: problem.f(x) * phi(x), -1, 1)[0]
    
    N = len(coordinates)
    matrix = np.zeros((N, N))
    b = np.zeros(N)
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            matrix[i, j] = bilinear_form(coordinates[i], coordinates[j])
        b[i] = dot(coordinates[i])

    coeffs = LA.solve(matrix, b)
    return lambda x: sum(coeffs[i] * coordinates[i](x) for i in range(N))

In [4]:
def collocation_method(problem: BoundaryValueProblem, N: int, nodes_type: Literal['chebroots', 'equidistant']) -> Function:
    assert problem.form == 2

    def get_coordinate_system() -> CoordinateSystem:
        system = []

        alpha_1, alpha_2, _ = problem.left_condition
        beta_1, beta_2, _ = problem.right_condition
        
        if N >= 1:
            matrix = [[-(alpha_1 + alpha_2), alpha_1],
                      [beta_1 + beta_2, beta_1]]
            b = [-(alpha_1 + 2 * alpha_2),
                 -(beta_1 + 2 * beta_2)]
            coeffs = np.append(list(reversed(LA.solve(matrix, b))), 1)
            system.append(Polynomial(coeffs))

        if N >= 2:
            matrix = [[-(alpha_1 + alpha_2), alpha_1],
                      [beta_1 + beta_2, beta_1]]
            b = [alpha_1 + 3 * alpha_2,
                 -(beta_1 + 3 * beta_2)]
            coeffs = np.append(list(reversed(LA.solve(matrix, b))), [0, 1])
            system.append(Polynomial(coeffs))

        if N >= 3:
            jacobi_coeff = Polynomial([1, 0, -1]) ** 2
            system += [jacobi_coeff * j for j in get_jacobi(N - 2, 2)]

        return system

    coordinates = get_coordinate_system()

    match nodes_type:
        case 'chebroots':
            nodes = cheb.chebroots(np.append(np.zeros(N), 1))
        case 'equidistant':
            nodes = np.linspace(-1, 1, N)

    def Lphi(phi: Polynomial) -> Function:
        return lambda x: problem.p(x) * phi.deriv(2)(x) + problem.q(x) * phi.deriv()(x) + problem.r(x) * phi(x)
        
    matrix = np.zeros((N, N))
    b = np.zeros(N)
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            matrix[i, j] = Lphi(coordinates[j])(nodes[i])
        b[i] = problem.f(nodes[i])

    coeffs = LA.solve(matrix, b)
    return lambda x: sum(coeffs[i] * coordinates[i](x) for i in range(N))

In [5]:
def solve_tridiagonal_system(matrix: lil_matrix, b: np.ndarray) -> np.ndarray:
    s = np.zeros(b.size)
    t = np.zeros(b.size)
    A, B, C, G = np.insert(matrix.diagonal(-1), 0, 0), -matrix.diagonal(), np.append(matrix.diagonal(1), 0), b

    for i in range(b.size):
        s[i] = C[i] / (B[i] - A[i] * s[i - 1])
        t[i] = (A[i] * t[i - 1] - G[i]) / (B[i] - A[i] * s[i - 1])
    
    y = np.zeros(b.size)
    y[-1] = t[-1]

    for i in reversed(range(b.size - 1)):
        y[i] = s[i] * y[i + 1] + t[i]

    return y


def solve_with_grid(problem: BoundaryValueProblem, N: int) -> tuple[np.ndarray, np.ndarray]:
    size = N + 1
    points, h = np.linspace(*problem.segment, size, retstep=True)

    matrix = lil_matrix((size, size))
    b = np.zeros(size)

    for i in range(1, N):
        x_i = points[i]
        matrix[i, i - 1] = -problem.p(x_i) / (h ** 2) - problem.q(x_i) / (2 * h)
        matrix[i, i] = problem.p(x_i) * 2 / (h ** 2) + problem.r(x_i)
        matrix[i, i + 1] = -problem.p(x_i) / (h ** 2) + problem.q(x_i) / (2 * h)
        b[i] = problem.f(x_i)

    alpha_1, alpha_2, alpha = problem.left_condition
    beta_1, beta_2, beta = problem.right_condition

    matrix[0, :3] = alpha_1 - alpha_2 * ((-3 / 2) / h), -alpha_2 * (2 / h), -alpha_2 * ((-1 / 2) / h)
    b[0] = alpha

    matrix[N, N - 2:] = beta_2 * ((1 / 2) / h), beta_2 * (-2 / h), beta_1 + beta_2 * ((3 / 2) / h)
    b[N] = beta

    coefficient_0, coefficient_N = matrix[0, 2] / matrix[1, 2], matrix[N, N - 2] / matrix[N - 1, N - 2]
    matrix[0] -= coefficient_0 * matrix[1]
    matrix[N] -= coefficient_N * matrix[N - 1]
    b[0] -= coefficient_0 * b[1]
    b[N] -= coefficient_N * b[N - 1]

    return points, solve_tridiagonal_system(matrix, b)


def solve_bvp(problem: BoundaryValueProblem, accuracy: float, start_n: int = 10) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    assert problem.form == 3
    
    n = start_n
    grid_errors = []

    _, previous_solution = solve_with_grid(problem, n)
    while True:
        n *= 2
        points, current_solution = solve_with_grid(problem, n)
        
        errors = np.zeros(current_solution.size)
        for i in range(0, errors.size, 2):
            errors[i] = (current_solution[i] - previous_solution[i // 2]) / 3
        for i in range(1, errors.size - 1, 2):
            errors[i] = (errors[i - 1] + errors[i + 1]) / 2

        error = max(abs(errors))
        grid_errors.append((n, error))

        current_solution += errors
        if error < accuracy:
            break

        previous_solution = current_solution
    return points, current_solution, np.array(grid_errors)

In [6]:
POINTS = np.linspace(-1, 1, 1001)

def ritz_method_plot(problem: BoundaryValueProblem, coordinates: CoordinateSystem, start_N: int = 3, stop_N: int = 7) -> go.Figure:
    fig = go.Figure()

    for n in range(start_N, stop_N + 1):
        u = ritz_method(problem, coordinates[:n])
        fig.add_trace(go.Scatter(x=POINTS, y=np.vectorize(u)(POINTS), name=f'N = {n}'))
    
    fig.update_layout(title='Метод Ритца')
    fig.update_xaxes(hoverformat='.6f')
    fig.update_yaxes(hoverformat='.6f')
    return fig


def collocation_method_plot(problem: BoundaryValueProblem, N: int = 7) -> go.Figure:
    fig = go.Figure()

    u = collocation_method(problem, N, 'chebroots')
    fig.add_trace(go.Scatter(x=POINTS, y=np.vectorize(u)(POINTS), name=f'узлы Чебышева'))

    u = collocation_method(problem, N, 'equidistant')
    fig.add_trace(go.Scatter(x=POINTS, y=np.vectorize(u)(POINTS), name=f'равноотстоящие узлы'))

    fig.update_layout(title='Метод коллокации')
    fig.update_xaxes(hoverformat='.6f')
    fig.update_yaxes(hoverformat='.6f')
    return fig


def get_grid_method_line(problem: BoundaryValueProblem, accuracy: float = 10 ** -6) -> go.Scatter:
    grid_points, grid_solution, _ = solve_bvp(problem, accuracy)
    return go.Scatter(x=grid_points, y=grid_solution, name='метод сеток')

In [7]:
p = lambda x: (2 + x) / (3 + x)
r = lambda x: 1 + sin(x)
f = lambda x: 1 - x
segment = (-1, 1)
l_condition = (0, -1, 0)
r_condition = (1, 1, 0)

dp = lambda x: 1 / (3 + x) ** 2
grid_problem = BoundaryValueProblem(3, p, lambda x: -dp(x), r, f, segment, l_condition, r_condition)
grid_method_line = get_grid_method_line(grid_problem)

ritz_problem = BoundaryValueProblem(1, p, None, r, f, segment, l_condition, r_condition)
N, k = 7, 1
coordinates = get_jacobi(N, k)
ritz_plot = ritz_method_plot(ritz_problem, coordinates, stop_N=N)
ritz_plot.add_trace(grid_method_line)
ritz_plot.show()

collocation_problem = BoundaryValueProblem(2, lambda x: -p(x), lambda x: -dp(x), r, f, segment, l_condition, r_condition)
collocation_plot = collocation_method_plot(collocation_problem, N=11)
collocation_plot.add_trace(grid_method_line)
collocation_plot.show()